#### SBERT
- BERT 모델 : 문장 이해용 Encoder
    - 문장 쌍 비교
- SBERT 모델 : 문장 의미 임베딩 
    - 벡터의 비교용 

In [ ]:
# 라이브러리 설치 
# !pip install sentence-transformers

In [2]:
import torch 
from sentence_transformers import SentenceTransformer, util

In [ ]:
# 모델을 로드 -> 두개의 문장을 비교(코사인 유사도)
# 다목적 한국 SBERT
model_name = 'jhgan/ko-sroberta-multitask'
# 문장 유사도 특화 
model_name2 = 'BM-K/KoSimCSE-roberta-multitask'

sbert = SentenceTransformer(model_name)
sbert2 = SentenceTransformer(model_name2)

In [5]:
# 최대 토큰의 길이를 설정 
sbert.max_seq_length = 256
sbert2.max_seq_length = 256

In [6]:
# 두개의 문장을 비교 
doc1 = "이 카메라는 색감이 자연스럽고 배터리도 오래간다"
doc2 = "배터리 성능이 좋고 사진 품질이 뛰어나다"

In [7]:
# 두개의 문장을 임베딩 -> 코사인 유사도 계산 
# sbert인 경우 
with torch.inference_mode():
    emb1 = sbert.encode(doc1, convert_to_tensor=True, normalize_embeddings=True)
    emb2 = sbert.encode(doc2, convert_to_tensor=True, normalize_embeddings=True)
# 코사인 유사도 계산
cos_sim = util.cos_sim(emb1, emb2).item()
print("유사도 : ", round(cos_sim, 4))

유사도 :  0.6948


In [8]:
# 두개의 문장을 임베딩 -> 코사인 유사도 계산 
# sbert2인 경우 
with torch.inference_mode():
    emb1 = sbert2.encode(doc1, convert_to_tensor=True, normalize_embeddings=True)
    emb2 = sbert2.encode(doc2, convert_to_tensor=True, normalize_embeddings=True)
# 코사인 유사도 계산
cos_sim = util.cos_sim(emb1, emb2).item()
print("유사도 : ", round(cos_sim, 4))

유사도 :  0.734


In [9]:
emb1.shape

torch.Size([768])

In [10]:
sentences = [
    '삼성전자 주가가 올랐다', 
    "코스피가 상승 마감했다", 
    '비가 많이 와서 항공편이 취소됬다'
]

with torch.inference_mode():
    embs = sbert2.encode(sentences, convert_to_tensor=True, 
                         normalize_embeddings=True)

# 3개의 문장에서의 유사도를 확인 
sim_metrix = util.cos_sim(embs, embs)
print(sim_metrix)

tensor([[1.0000, 0.3585, 0.0140],
        [0.3585, 1.0000, 0.1416],
        [0.0140, 0.1416, 1.0000]])


In [11]:
new_sentence = "증시가 강세였다"
# 임베딩 
new_emb = sbert2.encode(new_sentence, convert_to_tensor=True, 
                        normalize_embeddings=True)
# 유사도가 높은 상위의 n개 확인 
top_n = 2
hits = torch.topk(
    util.cos_sim(new_emb, embs).squeeze(0), k = top_n
)
hits

torch.return_types.topk(
values=tensor([0.6161, 0.6014]),
indices=tensor([0, 1]))

In [12]:
for score, idx in zip( hits.values.tolist(), hits.indices.tolist() ):
    print(f"{sentences[idx]} | score : {round(score, 3)}")

삼성전자 주가가 올랐다 | score : 0.616
코스피가 상승 마감했다 | score : 0.601


### 연습 
- ratings_train.txt 파일을 로드 
- 결측치 제거 
- document 컬럼의 문자 정규화(특수문자 제거, 2칸 이상의 공백 제거, 좌우 공백 제거) 
- 중복 document 제거 , 글자의 수가 1개 이하인 행은 제거  
- DataFrame에서 sample(n = 10000, random_state=42)로 임의의 데이터를 추출하여 저장 (head() -> 상위 데이터 | tail() -> 하위 데이터 | sample() -> 무작위 데이터) 
- train, test 셋으로 8:2 로 데이터분할
- sbert 모델은 'BM-K/KoSimCSE-roberta-multitask'을 이용
- Dataset을 정의 (Trainer 이용하지 않고 Dataset과 DataLoader 사용)
    - `__init__(self, document, labels)`
        - 입력받은 document와 labels를 document는 SBERT 모델을 이용하여 인코딩 
        - labels 데이터를 tensor형태로 변환 
    - `__len__(self)` 함수는 라벨의 길이를 되돌려준다
    - `__getitem__(self, idx)` 함수는 인코딩된 데이터[idx], label[idx]를 되돌려준다
- Dataset를 train, test를 이용해서 Dataset을 생성 
- DataLoarder를 이용하여 배치의 사이즈는 128 shuffle은 True 구성한다. 

In [13]:
import re
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

In [14]:
# 데이터 로드 
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [16]:
# 결측치를 제거 
df.dropna(subset='document', inplace=True)

In [17]:
def normalize(text):
    text = re.sub(r'[^가-힣0-9a-zA-Z\s\.]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [18]:
df['document'] = df['document'].map(normalize)

In [20]:
# 중복 데이터를 제거 
df.drop_duplicates(subset='document', inplace=True)

In [22]:
len(df)

144734

In [23]:
# 문자열의 길이가 1 이하는 제거 -> 1초과인 데이터만 확인 
flag = df['document'].str.len() > 1
df = df.loc[flag,]

In [26]:
# 랜덤한 데이터 10000개추출 sample()
df = df.sample(n = 10000, random_state=42).reset_index(drop=True)

In [27]:
# train, test 분할 (8:2)
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)

In [28]:
train_df['label'].value_counts()

label
0    4006
1    3994
Name: count, dtype: int64

In [29]:
model_name3 = 'BM-K/KoSimCSE-roberta-multitask'
sbert3 = SentenceTransformer(model_name3)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [30]:
# Dataset 정의 
class SBERTDataset(Dataset):
    # 생성자 함수 -> document, labels 받아와서 document 임베딩, labels는 tensor화
    def __init__(self, document, labels):
        # no_grad() -> 자동 미분 일시 정지 
        # inference_mode() -> 추론 모드 
        with torch.inference_mode():
            # 로드한 모델을 이용해서 encode 작업 
            # convet_to_tensor -> 결과값을 tensor로 받을것인가? (False : list)
            # normalize_embeddings -> L2 정규화 할것인가?
            self.emb = sbert3.encode(
                document, convert_to_tensor = True, normalize_embeddings=True
            )
        # labels를 tensor화 
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        # labels의 길이를 되돌려준다.
        return len(self.labels)
    def __getitem__(self, idx):
        return self.emb[idx], self.labels[idx]

In [31]:
# Dataset의 형태로 데이터프레임을 변환 
train_ds = SBERTDataset( train_df['document'].tolist(), train_df['label'].tolist() )
test_ds = SBERTDataset(test_df['document'].tolist(), test_df['label'].tolist())

In [32]:
# DataLoader를 이용해서 배치 사이즈만큼의 데이터를 생성 
train_dl = DataLoader(train_ds, batch_size = 128, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = 128, shuffle=True)